In [2]:
import os
import numpy as np
from dftpy.grid import DirectGrid
from dftpy.field import DirectField
from dftpy.functional import Functional, TotalFunctional
from dftpy.optimization import Optimization
from dftpy.constants import LEN_CONV, ENERGY_CONV
from dftpy.formats.vasp import read_POSCAR
from dftpy.td.propagator import Propagator
from dftpy.td.hamiltonian import Hamiltonian
from dftpy.utils.utils import calc_rho, calc_j
from dftpy.td.utils import initial_kick
from dftpy.mpi import pmi, sprint, mp
from dftpy.ions import Ions
#-----------------------------------------------------------------------
if pmi.size > 0:
    from mpi4py import MPI
    mp.comm = MPI.COMM_WORLD
#-----------------------------------------------------------------------
import ase
#drom ase.lattice.cubic import FaceCenteredCubic
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.io.trajectory import Trajectory
from ase.calculators.mixing import LinearCombinationCalculator, MixedCalculator, SumCalculator
from ase import units,Atoms

from ase.build import bulk, molecule
from ase.lattice.cubic import FaceCenteredCubic
from ase.md.langevin import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.io.trajectory import Trajectory
from ase.md.verlet import VelocityVerlet
from ase.calculators.eam import EAM

from dftpy.config.config import DefaultOption, OptionFormat, PrintConf
from dftpy.api.api4ase import DFTpyCalculator


In [3]:
np.random.seed(8888)

In [4]:
size = 1 
a = 4.24068463425528 
T = 600.0  # Kelvin
DATA='../DATA/'
#atoms = Atoms('Al', positions=[(a/2,a/2,a/2)], pbc=False, cell=[[a, 0, 0], [0, a, 0], [0, 0, a]])
atoms =  bulk('Al', 'fcc', a=a, cubic=True)

ions = Ions.from_ase(atoms)

In [5]:

PP_list = {'Al': DATA+'al.lda.recpot'}
grid = DirectGrid(ecut=20, lattice=ions.cell, mp=mp)
rho = DirectField(grid=grid)

PSEUDO = Functional(type='PSEUDO', grid=grid, ions=ions, PP_list=PP_list)
KE = Functional(type='KEDF', name='TFvW')
XC = Functional(type='XC', name='LDA')
HARTREE = Functional(type='HARTREE')
#
funcDict = {'KE' :KE, 'XC' :XC, 'HARTREE' :HARTREE, 'PSEUDO' :PSEUDO}
EnergyEvaluator = TotalFunctional(**funcDict)
opt = {'econv': 1e-12,'maxiter': 500}
#
rho[:] = ions.get_ncharges() / ions.cell.volume
optimizer = Optimization(EnergyEvaluator=EnergyEvaluator, optimization_method='TN',optimization_options=opt )

setting key: Al -> ../DATA/al.lda.recpot


In [6]:
calcBO = DFTpyCalculator(optimizer = optimizer, evaluator = EnergyEvaluator, rho = rho)

In [7]:
nas = 1

conf = DefaultOption()
conf["PATH"]["pppath"] = DATA
conf["PP"]["Al"] = "al.lda.recpot"
conf["OPT"]["method"] = "TN"
conf["OPT"]["econv"] = 1e-12
conf["KEDF"]["kedf"] = "TFvW"
conf["JOB"]["calctype"] = "Energy Force"
conf["OUTPUT"]["time"] = False
conf["TD"]["single_step"] = True
conf["TD"]["max_pc"] = 2
conf["TD"]["timestep"] = 0.041341374575751*nas
conf["TD"]["strength"] = 0.00 
conf["GRID"]["cplx"] = True
conf["GRID"]["gfull"] = True
conf["GRID"]["ecut"] =  540
#conf["GRID"]["nr"] = "16 16 16"
conf["PROPAGATOR"]["propagator"] = "crank-nicholson"
conf["PROPAGATOR"]["tol"] = 1e-14
conf["PROPAGATOR"]["atol"] = 1e-14

conf = OptionFormat(conf)
PrintConf(conf)


{
    "CASIDA": {
        "diagonalize": true,
        "numeig": null,
        "tda": false
    },
    "CELL": {
        "cellfile": "POSCAR",
        "elename": null,
        "format": null,
        "zval": null
    },
    "DENSITY": {
        "densityfile": null,
        "densityini": "heg",
        "densityoutput": null,
        "magmom": 0,
        "nspin": 1
    },
    "EXC": {
        "c_str": null,
        "libxc": [
            "lda_x",
            "lda_c_pz"
        ],
        "x_str": null,
        "xc": null
    },
    "GRID": {
        "cplx": true,
        "ecut": 540,
        "gfull": true,
        "maxprime": 13,
        "nr": null,
        "scale": 0.99,
        "spacing": 0.263884851105058
    },
    "INVERSION": {
        "rho_in": null,
        "v_out": null
    },
    "JOB": {
        "calctype": [
            "Energy",
            "Force"
        ],
        "task": "Optdensity"
    },
    "KEDF": {
        "alpha": null,
        "beta": null,
        "delta": null,

'{\n    "CASIDA": {\n        "diagonalize": true,\n        "numeig": null,\n        "tda": false\n    },\n    "CELL": {\n        "cellfile": "POSCAR",\n        "elename": null,\n        "format": null,\n        "zval": null\n    },\n    "DENSITY": {\n        "densityfile": null,\n        "densityini": "heg",\n        "densityoutput": null,\n        "magmom": 0,\n        "nspin": 1\n    },\n    "EXC": {\n        "c_str": null,\n        "libxc": [\n            "lda_x",\n            "lda_c_pz"\n        ],\n        "x_str": null,\n        "xc": null\n    },\n    "GRID": {\n        "cplx": true,\n        "ecut": 540,\n        "gfull": true,\n        "maxprime": 13,\n        "nr": null,\n        "scale": 0.99,\n        "spacing": 0.263884851105058\n    },\n    "INVERSION": {\n        "rho_in": null,\n        "v_out": null\n    },\n    "JOB": {\n        "calctype": [\n            "Energy",\n            "Force"\n        ],\n        "task": "Optdensity"\n    },\n    "KEDF": {\n        "alpha": 

In [8]:
calcEF = DFTpyCalculator(rho=rho,config=conf,optimizer = optimizer, evaluator = EnergyEvaluator, mp =mp, step = nas)

In [9]:
calcFF =  EAM(potential=DATA+'Al_zhou.eam.alloy')

FileNotFoundError: [Errno 2] No such file or directory: '../DATA/Al_zhou.eam.alloy'

In [11]:
calcA = [calcFF,calcBO,calcEF]
weights = [1.0,-1.0,1.0]

calcT= LinearCombinationCalculator(calcA,weights) 

atoms.calc = calcT

In [12]:
MaxwellBoltzmannDistribution(atoms, temperature_K = T, force_temp=True)

#dyn = Langevin(atoms, 0.001 * units.fs, temperature_K = T,  friction = 0.1)
dyn = VelocityVerlet(atoms, .001 * nas * units.fs )

step = 0
interval = 1
def printenergy(a=atoms):
    global step, interval
    epot = a.get_potential_energy() 
    ekin = a.get_kinetic_energy() 
    sprint(
            "Step={:<8d} Epot={:.5f} Ekin={:.5f} T={:.3f} Etot={:.5f} ".format(
            step, epot, ekin, ekin / (1.5 * units.kB), epot + ekin
        )
    )
    sprint(a.get_forces())
    step += interval

def check_stop():
    if os.path.isfile('dftpy_stopfile'): exit()


traj = Trajectory("testing.traj", "w", atoms)

dyn.attach(check_stop, interval=1)
dyn.attach(printenergy, interval=1)
dyn.attach(traj.write, interval=1)

dyn.run(10000)

Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -8.199458735952E+00     -8.199459E+00   1.053102E+00    1       1       1.206040E-02    
1       -8.424503649374E+00     -2.250449E-01   4.812652E-02    2       1       2.593756E-02    
2       -8.431858222657E+00     -7.354573E-03   2.445270E-03    5       1       4.825282E-02    
3       -8.432098935728E+00     -2.407131E-04   1.739177E-04    5       1       6.955218E-02    
4       -8.432122522005E+00     -2.358628E-05   9.065846E-06    6       1       8.758616E-02    
5       -8.432123012256E+00     -4.902511E-07   8.426012E-07    2       1       1.003754E-01    
6       -8.432123122652E+00     -1.103955E-07   5.362376E-08    6       1       1.252122E-01    
7       -8.432123127905E+00     -5.253385E-09   4.385821E-09    5       1       1.396940E-01    
8       -8.432123128224E+00     -3.189271E-10   3.445510E-10    3       1       1.549060E-01    
9       -8.432123128280E+00   


KeyboardInterrupt

